In [14]:
import praw
from prawcore.exceptions import RequestException, ResponseException, OAuthException
import pandas as pd
import time
import os

reddit = praw.Reddit(client_id='aC3FI1WNM2F5uaSSJF2rEw', 
                     client_secret='y7BQGM0bRxCVyPEh4oQnioS2rW57ZQ', 
                     user_agent='gryphonholder')


In [15]:
subreddit = reddit.subreddit('ADHD')

In [37]:
count = 0
posts = []

In [38]:
subreddit = reddit.subreddit('ADHD')
count = 0
posts = []
processed_ids = set()  # This will store the IDs of the processed posts

# Check if a count file exists and load the count from there
if os.path.exists('count.txt'):
    with open('count.txt', 'r') as f:
        count = int(f.read())

# Check if a posts file exists and load the posts from there
if os.path.exists('ADHDposts.csv'):
    df = pd.read_csv('ADHDposts.csv')
    posts = df.to_dict('records')

start_time = time.time()  # Initialize start_time

while count < 300000:  # Adjust the condition for the desired total count
    try:
        for submission in subreddit.new(limit=None):
            if submission.id in processed_ids:
                continue  # Skip this post if it has been processed before
            posts.append({
                "id": submission.id,
                "title": submission.title,
                "text": submission.selftext,
                "date": submission.created_utc,
                "upvotes": submission.score,
            })
            count += 1
            processed_ids.add(submission.id)  # Add the post ID to the set of processed IDs
            if count % 1000 == 0:
                end_time = time.time()  # End time after 1000 extractions
                print(f"Extraction count: {count}")
                rate = 1000 / (end_time - start_time)
                print(f"Extraction rate (last 1000): {rate:.2f} extractions/second")
                print(f"Extraction time (last 1000): {1000/rate:.2f} seconds taken")
                print(f"Title of the post: {submission.title}")
                start_time = time.time()  # Reset start_time for next 1000 extractions
    except (RequestException, OAuthException, ResponseException) as e:
        print(f"An error occurred: {e}")
        print("Sleeping for 60 seconds before trying again.")
        time.sleep(60)
    finally:
        # Save the count to a file
        with open('count.txt', 'w') as f:
            f.write(str(count))


KeyboardInterrupt: 

In [39]:

subreddit = reddit.subreddit("ADHD")  # replace with your subreddit

posts = []
start_time = time.time()

for count, submission in enumerate(subreddit.new(limit=10000), 1):
    posts.append({
        "id": submission.id,
        "title": submission.title,
        "text": submission.selftext,
        "score": submission.score
    })

    if count % 500 == 0:
        end_time = time.time()
        rate = 500 / (end_time - start_time)
        print(f"Extraction count: {count}")
        print(f"Extraction rate (last 500): {rate:.2f} extractions/second")
        print(f"Extraction time (last 500): {500/rate:.2f} seconds taken")
        print(f"Title of the last post: {submission.title}")
        start_time = time.time()

df = pd.DataFrame(posts)



Extraction count: 500
Extraction rate (last 500): 58.63 extractions/second
Extraction time (last 500): 8.53 seconds taken
Title of the last post: ADHD and Infidelity: Is there really a correlation? Or are people just making excuses?


In [40]:
# Save the posts to a CSV file
df = pd.DataFrame(posts)
df.to_csv('ADHD_10K_notcleaned.csv', index=False)

In [41]:
df

,id,title,text,score
0,149093g,Presentation differences,"Me and my husband have ADHD, he has inattentiv...",1
1,14903kr,Can't sleep racing thoughts,This is probably very common with many of us h...,1
2,1490208,Forgotten wife,My husband has diagnosed ADHD. For every anniv...,1
3,148ztb0,Need help regarding stimulants .,Hi i am 27m with adhd. I was prescribed with s...,1
4,148zqlr,30M medication and sex drive questions,Healthy active male Diagnosed at 30 after an i...,1
...,...,...,...,...
990,145600y,"Lysine, Tyrosine, or other amino acids for ADHD?",Had anyone had any luck taking Lysine or other...,3
991,1455yoz,I seems everything I exert myself to do for mo...,I have basically allowed myself to stop studyi...,6
992,1455t02,"After 4 years of gaslighting, i finally have m...",I've been a long time lurker and reading advic...,0
993,1455mhk,I have a drug screening for a new position in ...,Title.\n\nKind of freaking out a bit. This job...,4


In [31]:
duplicates = df[df.duplicated('title', keep=False)]
duplicate_counts = duplicates['title'].value_counts()



print(duplicate_counts)

Any podcast you would recommend ?                                                  78
My son is struggling socially                                                      39
i have problems?                                                                   39
What do you think about effecto?                                                   39
i have been diagnosed but it's not what i expected.                                39
                                                                                   ..
I possibly have adhd- and this possibility contributed to losing my job I think    16
TW impulsive urges                                                                 14
Do I potentially have ADHD?                                                         7
Any tips on how to absorb information when reading/listening to a book?             7
Racing Heart                                                                        7
Name: title, Length: 999, dtype: int64


In [21]:
df2

,id,title,text,url,date,upvotes,upvote_ratio
0,148wao8,My son is struggling socially,My son (8M) was diagnosed with ADHD 4 years ba...,https://www.reddit.com/r/ADHD/comments/148wao8...,1.686710e+09,1,1.00
1,148w58a,Anybody tried Seroquel?,"So I'll try to be brief, but I recently accept...",https://www.reddit.com/r/ADHD/comments/148w58a...,1.686710e+09,1,1.00
2,148w2dh,Considering legal action against psychiatrist ...,I’ve been going to my psychiatry practice for ...,https://www.reddit.com/r/ADHD/comments/148w2dh...,1.686709e+09,1,1.00
3,148w1fb,Hyperfixations,I've been wondering for a while what causes a ...,https://www.reddit.com/r/ADHD/comments/148w1fb...,1.686709e+09,1,1.00
4,148vxoj,Are Prasco and other “authorized generics” act...,Hey all. \n\nInsurance isn’t covering name bra...,https://www.reddit.com/r/ADHD/comments/148vxoj...,1.686709e+09,1,1.00
...,...,...,...,...,...,...,...
295,148402c,procrastination is slowing me down,So it is summer time and 2 days from now i wil...,https://www.reddit.com/r/ADHD/comments/148402c...,1.686623e+09,2,0.67
296,1483o90,"ADHD- I feel like I've tried everything, yet I...","Hello, \n\n\nI've been diagnosed with ADHD fo...",https://www.reddit.com/r/ADHD/comments/1483o90...,1.686622e+09,1,0.60
297,1483m81,I want off My meds when I turn 16 how do I con...,I am a 15M turning 16 in a couple of months an...,https://www.reddit.com/r/ADHD/comments/1483m81...,1.686621e+09,0,0.50
298,1483b10,"No migraines, just a numb feeling",I’ve just passed over a month on Vyvanse and u...,https://www.reddit.com/r/ADHD/comments/1483b10...,1.686620e+09,0,0.50
